In [33]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
#data_df = pd.read_csv("/content/drive/MyDrive/house-prices-advanced-regression-techniques.zip (Unzipped Files)/train.csv")
data_df = pd.read_csv("/content/drive/MyDrive/house-prices-advanced-regression-techniques/train.csv")

#Preprocessing

Making a data frame just for PCA and TSNE, and deleting the rows that don't have just numbers in them

In [38]:
for_model = data_df.dropna()

Finding out outliers of columns with large variation of data


In [51]:
#applied get_dummies to all the columns with obj type data
cols_to_encode = ['MSZoning', 'Street', 'Alley','LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition', 'Heating']
ohe = pd.get_dummies(data_df[cols_to_encode])
data_df = pd.concat([data_df, ohe], axis=1)

In [52]:
#dropping all the columns we one hot vectorized
clean_df = data_df.drop(columns = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition', 'Heating'])

In [53]:
#combined all the odd columns together
clean_df = clean_df.dropna()

In [54]:
clean = clean_df
clean_df.dropna()

,Id,MSSubClass,LotFrontage,LotArea,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall
0,1,60,65.0,8450,5,2003,2003,196.0,706,0,...,0,0,1,0,0,1,0,0,0,0
1,2,20,80.0,9600,8,1976,1976,0.0,978,0,...,0,0,1,0,0,1,0,0,0,0
2,3,60,68.0,11250,5,2001,2002,162.0,486,0,...,0,0,1,0,0,1,0,0,0,0
3,4,70,60.0,9550,5,1915,1970,0.0,216,0,...,0,0,0,0,0,1,0,0,0,0
4,5,60,84.0,14260,5,2000,2000,350.0,655,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,62.0,7917,5,1999,2000,0.0,0,0,...,0,0,1,0,0,1,0,0,0,0
1456,1457,20,85.0,13175,6,1978,1988,119.0,790,163,...,0,0,1,0,0,1,0,0,0,0
1457,1458,70,66.0,9042,9,1941,2006,0.0,275,0,...,0,0,1,0,0,1,0,0,0,0
1458,1459,20,68.0,9717,6,1950,1996,0.0,49,1029,...,0,0,1,0,0,1,0,0,0,0


In [55]:
count = 0
for col in clean.columns:
    count = count + 1
    print(col)
print(count)

Id
MSSubClass
LotFrontage
LotArea
OverallCond
YearBuilt
YearRemodAdd
MasVnrArea
BsmtFinSF1
BsmtFinSF2
BsmtUnfSF
TotalBsmtSF
1stFlrSF
2ndFlrSF
LowQualFinSF
GrLivArea
BsmtFullBath
BsmtHalfBath
FullBath
HalfBath
BedroomAbvGr
KitchenAbvGr
TotRmsAbvGrd
Fireplaces
GarageYrBlt
GarageCars
GarageArea
WoodDeckSF
OpenPorchSF
EnclosedPorch
3SsnPorch
ScreenPorch
PoolArea
MiscVal
MoSold
YrSold
SalePrice
MSZoning_C (all)
MSZoning_FV
MSZoning_RH
MSZoning_RL
MSZoning_RM
Street_Grvl
Street_Pave
Alley_Grvl
Alley_Pave
LotShape_IR1
LotShape_IR2
LotShape_IR3
LotShape_Reg
LandContour_Bnk
LandContour_HLS
LandContour_Low
LandContour_Lvl
Utilities_AllPub
Utilities_NoSeWa
LotConfig_Corner
LotConfig_CulDSac
LotConfig_FR2
LotConfig_FR3
LotConfig_Inside
LandSlope_Gtl
LandSlope_Mod
LandSlope_Sev
Neighborhood_Blmngtn
Neighborhood_Blueste
Neighborhood_BrDale
Neighborhood_BrkSide
Neighborhood_ClearCr
Neighborhood_CollgCr
Neighborhood_Crawfor
Neighborhood_Edwards
Neighborhood_Gilbert
Neighborhood_IDOTRR
Neighborhood_Mea

In [56]:
covariance = clean_df.cov()
covariance.drop(covariance.columns.difference(['SalePrice']), 1, inplace=True)
sale_price_cov = covariance['SalePrice']

<ipython-input-56-8234018964d5>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  covariance.drop(covariance.columns.difference(['SalePrice']), 1, inplace=True)


In [57]:
data_df.cov()

<ipython-input-57-ea462e32018f>:1: FutureWarning: The default value of numeric_only in DataFrame.cov is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data_df.cov()


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall
Id,177755.000000,198.968472,-108.390207,-1.398197e+05,-16.539068,5.915696,-161.881768,-191.474297,-3842.687941,-966.102125,...,-0.343386,0.238520,2.570254,-2.447224,4.054147e-01,-2.356408,-0.039068,0.747430,0.779986,0.462646
MSSubClass,198.968472,1789.338306,-404.439075,-5.901742e+04,1.908771,-2.792117,35.581337,35.439934,175.837842,-1347.355751,...,0.114622,0.004835,0.395533,-0.604632,-2.528942e-02,-0.297661,0.219910,0.018313,0.021388,0.063339
LotFrontage,-108.390207,-404.439075,589.749169,8.199814e+04,8.673047,-1.573935,95.108483,45.744675,871.207424,2635.952669,...,-0.043750,0.049251,-0.740842,0.935671,0.000000e+00,0.021207,0.030084,0.007250,-0.025083,-0.033458
LotArea,-139819.667238,-59017.418634,81998.138404,9.962565e+07,1460.548271,-62.602871,4289.113242,2841.343594,188401.408828,974690.774209,...,8.082977,-12.512379,21.882767,63.236114,-2.666092e+00,-116.487664,140.939064,-11.121862,-4.238284,-6.425163
OverallQual,-16.539068,1.908771,8.673047,1.460548e+03,1.912679,-0.141484,23.906215,15.723417,103.012054,151.176998,...,-0.005615,-0.004103,-0.076065,0.125144,-2.124274e-03,0.022740,-0.006023,-0.008701,0.001235,-0.007126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Heating_GasA,-2.356408,-0.297661,0.021207,-1.164877e+02,0.022740,0.010563,0.957896,0.508365,1.818007,3.886546,...,0.000180,-0.000385,-0.002565,0.001878,-6.703785e-04,0.021452,-0.012067,-0.004693,-0.001341,-0.002682
Heating_GasW,-0.039068,0.219910,0.030084,1.409391e+02,-0.006023,-0.001615,-0.565333,-0.199166,-1.051919,-0.264232,...,-0.000101,0.000516,0.001529,-0.001056,-8.450149e-06,-0.012067,0.012185,-0.000059,-0.000017,-0.000034
Heating_Grav,0.747430,0.018313,0.007250,-1.112186e+01,-0.008701,-0.004131,-0.208961,-0.167279,-0.500205,-1.889978,...,-0.000039,-0.000066,0.000176,-0.000411,-3.286169e-06,-0.004693,-0.000059,0.004775,-0.000007,-0.000013
Heating_OthW,0.779986,0.021388,-0.025083,-4.238284e+00,0.001235,-0.000789,-0.105233,-0.022434,-0.142916,-0.211980,...,-0.000011,-0.000019,0.000246,-0.000117,-9.389054e-07,-0.001341,-0.000017,-0.000007,0.001369,-0.000004


In [58]:
data_df['SalePrice']

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [59]:
covariance = data_df.cov()
covariance.drop(covariance.columns.difference(['SalePrice']), 1, inplace=True)
covariance

<ipython-input-59-47431be0faa3>:1: FutureWarning: The default value of numeric_only in DataFrame.cov is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  covariance = data_df.cov()
<ipython-input-59-47431be0faa3>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  covariance.drop(covariance.columns.difference(['SalePrice']), 1, inplace=True)


,SalePrice
Id,-7.340732e+05
MSSubClass,-2.832337e+05
LotFrontage,7.124262e+05
LotArea,2.092111e+08
OverallQual,8.690413e+04
...,...
Heating_GasA,1.076627e+03
Heating_GasW,-1.762869e+02
Heating_Grav,-5.068872e+02
Heating_OthW,-7.562878e+01


In [60]:
covariance = abs(covariance)
print(covariance)

                 SalePrice
Id            7.340732e+05
MSSubClass    2.832337e+05
LotFrontage   7.124262e+05
LotArea       2.092111e+08
OverallQual   8.690413e+04
...                    ...
Heating_GasA  1.076627e+03
Heating_GasW  1.762869e+02
Heating_Grav  5.068872e+02
Heating_OthW  7.562878e+01
Heating_Wall  2.435125e+02

[545 rows x 1 columns]


In [61]:
covariance.sort_values(by=['SalePrice'], ascending=False)

,SalePrice
SalePrice,6.311111e+09
LotArea,2.092111e+08
GrLivArea,2.958187e+07
TotalBsmtSF,2.138442e+07
1stFlrSF,1.860663e+07
...,...
RoofStyle_Mansard,1.692509e+00
RoofMatl_Metal,6.313885e-01
RoofMatl_Metal,6.313885e-01
BsmtFinType2_GLQ,5.848235e-01


In [66]:
covariance.index.name = 'Name'

In [68]:
covariance

,SalePrice
Name,
Id,7.340732e+05
MSSubClass,2.832337e+05
LotFrontage,7.124262e+05
LotArea,2.092111e+08
OverallQual,8.690413e+04
...,...
Heating_GasA,1.076627e+03
Heating_GasW,1.762869e+02
Heating_Grav,5.068872e+02


In [70]:
covariance = covariance.reset_index()

In [71]:
top_100_name = covariance.sort_values(by='SalePrice', ascending=False)['Name'].head(100)
print(top_100_name)

37            SalePrice
3               LotArea
16            GrLivArea
12          TotalBsmtSF
13             1stFlrSF
             ...       
124       RoofStyle_Hip
378       RoofStyle_Hip
169    MasVnrType_Stone
423    MasVnrType_Stone
122     RoofStyle_Gable
Name: Name, Length: 100, dtype: object


In [81]:
top_100_names = covariance.sort_values(by='SalePrice', ascending=False)['Name'].head(100)
if not top_100_name.empty:
    top_100_names_df = pd.DataFrame({'Name': top_100_names})
    top_100_names_df.reset_index(drop=True, inplace=True)
    print(top_100_names_df)
else:
    print("No data available for top 100 names.")

                Name
0          SalePrice
1            LotArea
2          GrLivArea
3        TotalBsmtSF
4           1stFlrSF
..               ...
95     RoofStyle_Hip
96     RoofStyle_Hip
97  MasVnrType_Stone
98  MasVnrType_Stone
99   RoofStyle_Gable

[100 rows x 1 columns]
